In [ ]:
!pip install -qU langgraph==0.2.45 langchain-google-genai==2.0.4

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install pytesseract

In [ ]:
!pip install google-generativeai


In [ ]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain.tools import Tool
from langchain.schema import AIMessage, HumanMessage

# ✅ Set up Gemini API Key
os.environ["GOOGLE_API_KEY"] = ""  # Replace with your actual key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# ✅ Load Gemini Model in LangChain
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# ✅ Define State for Medical Chatbot
class MedicalChatState(TypedDict):
    """State representation of the medical chatbot session."""
    messages: Annotated[list, add_messages]  # Stores chat history
    symptoms: list[str]  # Stores symptoms reported by user
    diagnoses: list[str]  # Stores possible diagnoses
    finished: bool  # Indicates if the session is complete

# ✅ System Instructions for Chatbot
MEDBOT_SYSTINT = AIMessage(
    content=(
        "You are MedBot, an AI medical assistant. You help users by answering health-related queries, "
        "analyzing symptoms, and providing preliminary diagnoses. "
        "You do NOT prescribe medications or provide treatment recommendations. "
        "For serious conditions, always advise users to consult a medical professional."
    )
)

WELCOME_MSG = "Welcome to MedBot! Please describe your symptoms or ask a medical question."


In [ ]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
def chatbot(state:MedicalChatState)->MedicalChatState:
  """The chatbot itself.A simple wrapper around model's own chat interface"""
  message_history=[MEDBOT_SYSTINT]+state["messages"]
  return {"messages": [llm.invoke(message_history)]}

graph_builder=StateGraph(MedicalChatState)
graph_builder.add_node("medicalchatbot",chatbot)

graph_builder.add_edge(START,"medicalchatbot")

chat_graph=graph_builder.compile()


In [ ]:
from IPython.display import Markdown, display

mermaid_code = chat_graph.get_graph().draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


In [ ]:
from pprint import pprint
user_msg="Hello,what can you do?"
state=chat_graph.invoke({"messages":[user_msg]})
for msg in state["messages"]:
   print(f"{type(msg).__name__}:{msg.content}")



In [ ]:
import os
import pandas as pd
import google.generativeai as genai
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain.schema import AIMessage, HumanMessage
from typing import Annotated, Literal
from typing_extensions import TypedDict

In [ ]:
csv_path = "/content/processed_medquad.csv"
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ Dataset file not found: {csv_path}")

df = pd.read_csv(csv_path)
df = df.dropna(subset=["Question_Sentences", "Answer_Sentences"])  # Drop missing values

# ✅ Convert rows to LangChain Document objects
docs_to_add = [
    Document(
        page_content=str(row["Question_Sentences"]).strip(),
        metadata={
            "answer": str(row["Answer_Sentences"]).strip(),
            "qtype": str(row["qtype"]).strip(),
        }
    )
    for _, row in df.iterrows()
]

# ✅ Load Hugging Face Sentence Transformer Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Initialize Chroma Vector Store
chroma_db_path = "./chroma_db"
vector_store = Chroma(persist_directory=chroma_db_path, embedding_function=embedding_model)

# ✅ Add Documents to Vector Store (only if empty)
if len(vector_store.get()["ids"]) == 0:
    vector_store.add_documents(docs_to_add)
    print("✅ Medical QA dataset added to Chroma vector store!")
else:
    print("✅ Chroma vector store already populated.")


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [ ]:
from langchain_core.messages.ai import AIMessage

def human_node(state:MedicalChatState ) ->MedicalChatState:
  last_msg=state["messages"][-1]
  print("Model",last_msg.content)

  user_input=input("User: ")

  if user_input in {"q","quit","goodbye"}:
    state["finished"] =True

  return state| {"messages":[("user",user_input)]}

def chatbot_with_welcome_msg(state: MedicalChatState) -> MedicalChatState:
  if  state["messages"]:
    new_output=llm.invoke([MEDBOT_SYSTINT]+state["messages"])
  else:
    new_output=AIMessage(content=WELCOME_MSG)

  return state | {"messages":[new_output]}


graph_builder=StateGraph(MedicalChatState)

graph_builder.add_node("chatbot",chatbot_with_welcome_msg)
graph_builder.add_node("human",human_node)

graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot","human");


In [ ]:
from typing import Literal

def maybe_exit_human_node(state:MedicalChatState) ->Literal["chatbot","__end__"]:
  if state.get("finished",False):
    return END
  else:
    return "chatbot"

# Move these lines outside the function to make them global
graph_builder.add_conditional_edges("human",maybe_exit_human_node)
chat_with_human_graph=graph_builder.compile()

In [ ]:
state=chat_with_human_graph.invoke({"messages":[]})

In [ ]:

from langchain_core.tools import tool
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, START, END
from langchain.llms import OpenAI  # Only if you use it later
from langchain.schema import HumanMessage, AIMessage, BaseMessage
from typing import Literal, Annotated, TypedDict, List

# Define the MedicalChatState schema
class MedicalChatState(TypedDict):
    messages: List[BaseMessage]
    finished: bool

# ✅ Load pre-trained embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Load the existing Chroma vector store
vector_store = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

# ✅ Tool for retrieving medical answers
@tool
def retrieve_medical_answer(query: str) -> str:
    """
    Retrieves the most relevant medical answer from the Chroma vector store.
    If no relevant answer is found, returns an empty response.
    """
    results = vector_store.similarity_search(query, k=1)
    if results:
        best_match = results[0].metadata.get("answer", "").strip()
        if best_match:
            return f"🔹 **Best Matched Answer:** {best_match}"
    return ""


# ✅ Human node for user input
def human_node(state: MedicalChatState) -> MedicalChatState:
    if not state.get("messages"):
        print("🤖 MedBot: Welcome to MedBot! Ask me any medical question.")
    else:
        last_msg = state["messages"][-1]
        print("🤖 MedBot:", last_msg.content)

    user_input = input("👤 User: ")

    # Handle exit
    if user_input.lower() in {"q", "quit", "goodbye"}:
        return {**state, "finished": True}

    # Append new user message
    messages = state.get("messages", [])
    return {**state, "messages": messages + [HumanMessage(content=user_input)], "finished": False}


# ✅ Chatbot node with RAG fallback
def chatbot_with_rag_fallback(state: MedicalChatState) -> MedicalChatState:
    if not state["messages"]:
        return {**state, "messages": [AIMessage(content="Welcome to MedBot! Ask me any medical question.")]}

    query = state["messages"][-1].content
    retrieved_answer = retrieve_medical_answer.invoke(query)

    if retrieved_answer:
        return {**state, "messages": state["messages"] + [AIMessage(content=retrieved_answer)]}
    else:
        # Placeholder fallback response
        fallback_response = "I'm sorry, I couldn't find a direct answer. Please consult a medical professional."
        return {**state, "messages": state["messages"] + [AIMessage(content=fallback_response)]}


# ✅ Route control
def route_from_chatbot(state: MedicalChatState) -> Literal["human", "chatbot"]:
    return "human"


def decide_next_step(state: MedicalChatState) -> Literal["chatbot", "__end__"]:
    if state.get("finished", False):
        print("👋 Goodbye! Take care.")
        return END
    return "chatbot"


# ✅ Build the LangGraph
graph_builder = StateGraph(MedicalChatState)

# Add nodes
graph_builder.add_node("human", human_node)
graph_builder.add_node("chatbot", chatbot_with_rag_fallback)

# Define edges
graph_builder.set_entry_point("human")
graph_builder.add_conditional_edges("human", decide_next_step)
graph_builder.add_conditional_edges("chatbot", route_from_chatbot)

# ✅ Compile and run the graph
medical_chat_graph = graph_builder.compile()

# ✅ Run the chatbot
state = {"messages": [], "finished": False}
medical_chat_graph.invoke(state)


In [ ]:
!pip install tesseract

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import pytesseract
from PIL import Image
import google.generativeai as genai
import os

# Set up Gemini API
genai.configure(api_key="")

### 📌 1️⃣ Define Medical Chatbot State ###
class MedicalChatState(TypedDict):
    messages: list
    medical_report_text: str
    finished: bool

### 📌 2️⃣ Extract Text from Image (OCR) ###
def extract_text_from_image(image_path):
    try:
        if not os.path.exists(image_path):
            return "Error: Image file not found."

        image = Image.open(image_path)
        extracted_text = pytesseract.image_to_string(image).strip()

        return extracted_text if extracted_text else "No readable text found in the image."
    except Exception as e:
        return f"Error processing image: {str(e)}"

### 📌 3️⃣ Gemini 1.5 Flash AI Response ###
def chatbot_with_welcome_msg(state: MedicalChatState) -> MedicalChatState:
    if state["messages"]:
        last_msg = state["messages"][-1]

        # If medical text exists, AI should use it
        medical_text = state.get("medical_report_text", "")
        user_query = last_msg["content"]

        model = genai.GenerativeModel("gemini-1.5-flash")
        prompt = f"""
        You are a medical assistant AI. The extracted text from a patient's medical report is:

        {medical_text}

        The user has asked: "{user_query}"

        Provide a clear and medically accurate response based on the report.
        """
        response = model.generate_content(prompt)
        ai_response = AIMessage(content=response.text)
    else:
        ai_response = AIMessage(content="Welcome to the Medical Chatbot! Type 'img' to upload a medical report.")

    return state | {"messages": [ai_response]}

### 📌 4️⃣ Human Node with Image Handling ###
def human_node(state: MedicalChatState) -> MedicalChatState:
    last_msg = state["messages"][-1]
    print("\nModel:", last_msg.content)

    user_input = input("User (type text or enter 'img' to upload an image): ")

    if user_input.lower() in {"q", "quit", "goodbye"}:
        state["finished"] = True
        return state

    # If the user wants to upload an image
    if user_input.lower() == "img":
        image_path = input("Enter the image path: ")
        extracted_text = extract_text_from_image(image_path)

        print("\nExtracted Text:\n", extracted_text)

        return state | {"medical_report_text": extracted_text, "messages": [{"role": "user", "content": "Uploaded a medical report."}]}

    return state | {"messages": [{"role": "user", "content": user_input}]}

### 📌 5️⃣ Conditional Exit ###
def maybe_exit_human_node(state: MedicalChatState) -> Literal["chatbot", "__end__"]:
    return END if state.get("finished", False) else "chatbot"

### 📌 6️⃣ Build Conversation Flow ###
graph_builder = StateGraph(MedicalChatState)

graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "human")
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

chat_with_human_graph = graph_builder.compile()

### 📌 7️⃣ Start Chatbot ###
def main():
    state = chat_with_human_graph.invoke({"messages": [], "medical_report_text": ""})

if __name__ == "__main__":
    main()